In [36]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np



/Users/amitchoksi/Class/PyBer_Analysis
/Users/amitchoksi/Class/PyBer_Analysis


In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
print(lat_lngs)

In [4]:
from citipy import citipy

In [5]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [6]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)
cities

['troitsko-pechorsk',
 'bandarbeyla',
 'smithers',
 'souillac',
 'ushuaia',
 'port elizabeth',
 'kudahuvadhoo',
 'mao',
 'natchitoches',
 'chernyshevskiy',
 'payo',
 'victor harbor',
 'port alfred',
 'bengkulu',
 'severo-kurilsk',
 'nizhneyansk',
 'khatanga',
 'jamestown',
 'itarema',
 'clyde river',
 'punta arenas',
 'kapaa',
 'hobart',
 'cape town',
 'buala',
 'mahebourg',
 'airai',
 'arraial do cabo',
 'aligudarz',
 'palana',
 'butaritari',
 'santa isabel',
 'tulun',
 'nanortalik',
 'tuktoyaktuk',
 'paamiut',
 'rundu',
 'moree',
 'celestun',
 'nikolskoye',
 'praya',
 'grand centre',
 'mataura',
 'caravelas',
 'la ronge',
 'hithadhoo',
 'namibe',
 'albany',
 'rikitea',
 'betioky',
 'ilulissat',
 'chagda',
 'leningradskiy',
 'camacha',
 'ponta do sol',
 'morrinhos',
 'gazi',
 'cessnock',
 'upata',
 'touros',
 'cherskiy',
 'vao',
 'aquiraz',
 'sungaipenuh',
 'katsuura',
 'fort nelson',
 'busselton',
 'luderitz',
 'tasiilaq',
 'vaini',
 'pingdingshan',
 'norman wells',
 'moji das cruzes

In [33]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key
#

In [8]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=63773b8ed321df9b1aca6032f18b2605


In [9]:
# Create an endpoint URL for a city.
city_url = url + "&q=" + "Boston"
print(city_url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=63773b8ed321df9b1aca6032f18b2605&q=Boston


In [10]:
# Make a 'Get' request for the city weather.
city_weather = requests.get(city_url)
city_weather



<Response [200]>

In [11]:
if city_weather.status_code == 200:
    print(f"City Weather found.")
else:
    print(f"City weather not found.")

City Weather found.


In [12]:
# Get the JSON data.
boston_data = city_weather.json()

In [13]:
boston_data

{'coord': {'lon': -71.0598, 'lat': 42.3584},
 'weather': [{'id': 801,
   'main': 'Clouds',
   'description': 'few clouds',
   'icon': '02d'}],
 'base': 'stations',
 'main': {'temp': 68.9,
  'feels_like': 68.74,
  'temp_min': 60.94,
  'temp_max': 77.95,
  'pressure': 1009,
  'humidity': 69},
 'visibility': 10000,
 'wind': {'speed': 19.57, 'deg': 100},
 'clouds': {'all': 20},
 'dt': 1654378922,
 'sys': {'type': 2,
  'id': 2013408,
  'country': 'US',
  'sunrise': 1654333736,
  'sunset': 1654388192},
 'timezone': -14400,
 'id': 4930956,
 'name': 'Boston',
 'cod': 200}

In [14]:
# Get the JSON data.
boston_data = city_weather.json()

In [15]:
boston_data["sys"]

{'type': 2,
 'id': 2013408,
 'country': 'US',
 'sunrise': 1654333736,
 'sunset': 1654388192}

In [16]:
boston_data["sys"]["country"]

'US'

In [17]:
boston_data["dt"]

1654378922

In [18]:
boston_data["coord"]["lat"]

42.3584

In [19]:
boston_data["main"]["temp_max"]

77.95

In [20]:
lat = boston_data["coord"]["lat"]
lng = boston_data["coord"]["lon"]
max_temp = boston_data["main"]["temp_max"]
humidity = boston_data["main"]["humidity"]
clouds = boston_data["clouds"]["all"]
wind = boston_data["wind"]["speed"]
print(lat, lng, max_temp, humidity, clouds, wind)

42.3584 -71.0598 77.95 69 20 19.57


In [21]:
# Import the datetime module from the datetime library.
from datetime import datetime
# Get the date from the JSON file.
date = boston_data["dt"]
# Convert the UTC date to a date format with year, month, day, hours, minutes, and seconds.
datetime.utcfromtimestamp(date)

datetime.datetime(2022, 6, 4, 21, 42, 2)

In [22]:
datetime.utcfromtimestamp(date).strftime("%Y-%m-%d %H:%M:%S")

'2022-06-04 21:42:02'

In [23]:
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

In [24]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [25]:
print(cities)

['troitsko-pechorsk', 'bandarbeyla', 'smithers', 'souillac', 'ushuaia', 'port elizabeth', 'kudahuvadhoo', 'mao', 'natchitoches', 'chernyshevskiy', 'payo', 'victor harbor', 'port alfred', 'bengkulu', 'severo-kurilsk', 'nizhneyansk', 'khatanga', 'jamestown', 'itarema', 'clyde river', 'punta arenas', 'kapaa', 'hobart', 'cape town', 'buala', 'mahebourg', 'airai', 'arraial do cabo', 'aligudarz', 'palana', 'butaritari', 'santa isabel', 'tulun', 'nanortalik', 'tuktoyaktuk', 'paamiut', 'rundu', 'moree', 'celestun', 'nikolskoye', 'praya', 'grand centre', 'mataura', 'caravelas', 'la ronge', 'hithadhoo', 'namibe', 'albany', 'rikitea', 'betioky', 'ilulissat', 'chagda', 'leningradskiy', 'camacha', 'ponta do sol', 'morrinhos', 'gazi', 'cessnock', 'upata', 'touros', 'cherskiy', 'vao', 'aquiraz', 'sungaipenuh', 'katsuura', 'fort nelson', 'busselton', 'luderitz', 'tasiilaq', 'vaini', 'pingdingshan', 'norman wells', 'moji das cruzes', 'daoukro', 'santa cruz de tenerife', 'atuona', 'morros', 'hermanus', 

In [26]:
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Processing Record 1 of Set 1 | troitsko-pechorsk
Processing Record 2 of Set 1 | bandarbeyla
Processing Record 3 of Set 1 | smithers
Processing Record 4 of Set 1 | souillac
Processing Record 5 of Set 1 | ushuaia
Processing Record 6 of Set 1 | port elizabeth
Processing Record 7 of Set 1 | kudahuvadhoo
Processing Record 8 of Set 1 | mao
Processing Record 9 of Set 1 | natchitoches
Processing Record 10 of Set 1 | chernyshevskiy
Processing Record 11 of Set 1 | payo
Processing Record 12 of Set 1 | victor harbor
Processing Record 13 of Set 1 | port alfred
Processing Record 14 of Set 1 | bengkulu
Processing Record 15 of Set 1 | severo-kurilsk
Processing Record 16 of Set 1 | nizhneyansk
City not found. Skipping...
Processing Record 17 of Set 1 | khatanga
Processing Record 18 of Set 1 | jamestown
Processing Record 19 of Set 1 | itarema
Processing Record 20 of Set 1 | clyde river
Processing Record 21 of Set 1 | punta arenas
Processing Record 22 of Set 1 | kapaa
Processing Record 23 of Set 1 | hoba

Processing Record 37 of Set 4 | shetpe
Processing Record 38 of Set 4 | dingle
Processing Record 39 of Set 4 | pevek
Processing Record 40 of Set 4 | affery
Processing Record 41 of Set 4 | dubbo
Processing Record 42 of Set 4 | bjelovar
Processing Record 43 of Set 4 | brae
Processing Record 44 of Set 4 | lefka
Processing Record 45 of Set 4 | tura
Processing Record 46 of Set 4 | bathsheba
Processing Record 47 of Set 4 | saskylakh
Processing Record 48 of Set 4 | noumea
Processing Record 49 of Set 4 | comodoro rivadavia
Processing Record 50 of Set 4 | mehamn
Processing Record 1 of Set 5 | vagur
Processing Record 2 of Set 5 | seoul
Processing Record 3 of Set 5 | shelburne
Processing Record 4 of Set 5 | port lincoln
Processing Record 5 of Set 5 | fort dodge
Processing Record 6 of Set 5 | ariquemes
Processing Record 7 of Set 5 | nyurba
Processing Record 8 of Set 5 | iwanai
Processing Record 9 of Set 5 | kodiak
Processing Record 10 of Set 5 | meyungs
City not found. Skipping...
Processing Record

Processing Record 27 of Set 8 | bambari
Processing Record 28 of Set 8 | egvekinot
Processing Record 29 of Set 8 | gundorovskiy
Processing Record 30 of Set 8 | hovd
Processing Record 31 of Set 8 | hamilton
Processing Record 32 of Set 8 | sao miguel do araguaia
Processing Record 33 of Set 8 | wajima
Processing Record 34 of Set 8 | copiapo
Processing Record 35 of Set 8 | el dorado
Processing Record 36 of Set 8 | rudbar
Processing Record 37 of Set 8 | manjacaze
Processing Record 38 of Set 8 | passo de camaragibe
Processing Record 39 of Set 8 | sao joao do piaui
Processing Record 40 of Set 8 | mbanza-ngungu
Processing Record 41 of Set 8 | poum
Processing Record 42 of Set 8 | oussouye
Processing Record 43 of Set 8 | balakliya
Processing Record 44 of Set 8 | yumen
Processing Record 45 of Set 8 | constitucion
Processing Record 46 of Set 8 | banjar
Processing Record 47 of Set 8 | axim
Processing Record 48 of Set 8 | dawei
Processing Record 49 of Set 8 | pacific grove
Processing Record 50 of Set

Processing Record 20 of Set 12 | waipawa
Processing Record 21 of Set 12 | pavilosta
Processing Record 22 of Set 12 | teahupoo
Processing Record 23 of Set 12 | healdsburg
Processing Record 24 of Set 12 | mountain home
Processing Record 25 of Set 12 | shebunino
Processing Record 26 of Set 12 | mount darwin
Processing Record 27 of Set 12 | kavaratti
Processing Record 28 of Set 12 | bol
Processing Record 29 of Set 12 | gushi
Processing Record 30 of Set 12 | urusha
Processing Record 31 of Set 12 | tripoli
Processing Record 32 of Set 12 | hirata
Processing Record 33 of Set 12 | opuwo
Processing Record 34 of Set 12 | garissa
Processing Record 35 of Set 12 | shingu
Processing Record 36 of Set 12 | vengurla
Processing Record 37 of Set 12 | sur
Processing Record 38 of Set 12 | yellamanchili
City not found. Skipping...
Processing Record 39 of Set 12 | cockburn harbour
City not found. Skipping...
Processing Record 40 of Set 12 | petropavlovsk-kamchatskiy
Processing Record 41 of Set 12 | sumbe
Proc

In [38]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Troitsko-Pechorsk,62.7084,56.1964,44.80,63,0,6.67,RU,2022-06-04 21:46:47
1,Bandarbeyla,9.4942,50.8122,78.93,80,18,20.18,SO,2022-06-04 21:46:47
2,Smithers,54.7804,-127.1743,62.92,42,75,3.44,CA,2022-06-04 21:46:47
3,Souillac,-20.5167,57.5167,70.18,88,40,19.57,MU,2022-06-04 21:46:48
4,Ushuaia,-54.8000,-68.3000,37.06,60,20,11.50,AR,2022-06-04 21:46:48
5,Port Elizabeth,-33.9180,25.5701,62.89,81,75,17.00,ZA,2022-06-04 21:46:49
6,Kudahuvadhoo,2.6708,72.8944,83.12,73,20,13.44,MV,2022-06-04 21:46:49
7,Mao,39.8885,4.2658,70.36,94,40,13.80,ES,2022-06-04 21:46:50
8,Natchitoches,31.7668,-93.1335,90.45,51,0,3.44,US,2022-06-04 21:46:50
9,Chernyshevskiy,63.0128,112.4714,54.05,47,23,8.14,RU,2022-06-04 21:46:50


In [41]:
new_column_order = [ "City", "Country", "Date" ,"Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_column_order]

In [42]:
city_data_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Troitsko-Pechorsk,RU,2022-06-04 21:46:47,62.7084,56.1964,44.80,63,0,6.67
1,Bandarbeyla,SO,2022-06-04 21:46:47,9.4942,50.8122,78.93,80,18,20.18
2,Smithers,CA,2022-06-04 21:46:47,54.7804,-127.1743,62.92,42,75,3.44
3,Souillac,MU,2022-06-04 21:46:48,-20.5167,57.5167,70.18,88,40,19.57
4,Ushuaia,AR,2022-06-04 21:46:48,-54.8000,-68.3000,37.06,60,20,11.50
...,...,...,...,...,...,...,...,...,...
578,Usinsk,RU,2022-06-04 22:02:48,65.9939,57.5281,41.34,44,1,11.61
579,Correntina,BR,2022-06-04 22:02:49,-13.3433,-44.6367,73.58,41,15,5.50
580,Comendador,DO,2022-06-04 22:02:49,18.8775,-71.7028,86.43,51,95,7.87
581,Grindavik,IS,2022-06-04 22:02:50,63.8424,-22.4338,47.89,76,0,9.22


In [45]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")